In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def iqr(data):
    return data.quantile(0.75) - data.quantile(0.25)

file_path = 'data/dynamic_log_icra2024_3trees.json'

# Load the entire JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Create DataFrame
df = pd.json_normalize(data)

df = df.rename(columns={
    'result.time': 'Total Time',
    'result.total_path_length': 'Total Path Length',
    'result.timeout': 'Timeout'
})
df['Path Length per Visited Node'] = df['Total Path Length'] / df['result.n_visited']
df['Discoverable/Total Ratio'] = df['parameters.problem.n_discoverable'] / df['parameters.problem.n_total']
df['Visited/Total Ratio'] = df['result.n_visited'] / df['parameters.problem.n_total']

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/dynamic_log_icra2024_3trees.json'

In [ ]:


# Metrics to plot
metrics = ['Total Time', 'Path Length per Visited Node']#, 'Visited/Total Ratio', 'Timeout']

# Extract unique tree models and planners
tree_models = df['parameters.problem.tree_model'].unique()
planners = df['parameters.planner'].unique()

# Create subplots for each metric and tree model
fig, axs = plt.subplots(len(metrics), len(tree_models), figsize=(8, 6))

# For each metric and tree model, create a plot
for i, metric in enumerate(metrics):
    for j, tree_model in enumerate(tree_models):
        for planner in planners:
            df_planner_tree = df[df['parameters.planner'] == planner]
            df_planner_tree = df_planner_tree[df_planner_tree['parameters.problem.tree_model'] == tree_model].groupby('Discoverable/Total Ratio')[metric]

            # Calculate means and IQRs, but skip for Timeout as it's a proportion
            planner_means = df_planner_tree.mean().dropna()
            
            if metric != 'Timeout':
                planner_errors = df_planner_tree.apply(iqr).dropna()
                axs[i, j].errorbar(planner_means.index, planner_means.values, yerr=planner_errors.values, fmt='o-', label=planner)
            else:
                axs[i, j].plot(planner_means.index, planner_means.values, 'o-', label=planner)

        # Extract 'n_total' value for title
        n_total = df[df['parameters.problem.tree_model'] == tree_model]['parameters.problem.n_total'].iloc[0]

        # Set plot title, labels, and legend
        if i == 0:
            axs[i, j].set_title(f'n_total={n_total}')
        a     
        if i == len(metrics)-1:
            axs[i, j].set_xlabel('Discoverable/Total Ratio')
         
        if j == 0:
            axs[i, j].set_ylabel(metric)
        
        if (i,j) == (len(metrics)-1,len(tree_models)-1):
            axs[i, j].legend()

        # Set y-axis limit to start at 0
        axs[i, j].set_ylim(bottom=0)
        axs[i,j].grid()

plt.title('Hello')
        
# Improve layout and show plot
plt.tight_layout()
plt.show()

plt.savefig('lci_vs_fre.pdf')